# Try to find good pairs of images for comparison

In [1]:
import IPython.display as display
import ipywidgets as widgets
from astropy.io import fits
import math
import os
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from datetime import datetime

%matplotlib inline

In [2]:
data_path = '/media/mbentley/b182700e-67bc-47d5-9ee6-a98b2e94c806/rosetta_zoo/labels/'

## Load the data

In [3]:
images = pd.read_csv(os.path.join(data_path, 'osinac_metadata.csv.gz'))

In [4]:
images.start_time = pd.to_datetime(images.start_time)
images.stop_time = pd.to_datetime(images.stop_time)

In [5]:
len(images)

10345

# Produce some pairs

First try taking a few images pre-perihelion and comparing to after - this is not the only relevant set of comparisons, but let's give it a try:

In [6]:
t_perihelion = pd.Timestamp('2015-08-13T02:03:00Z').to_datetime64()

In [7]:
pre = images[images.start_time<t_perihelion]
post = images[images.start_time>=t_perihelion]

In [8]:
len(pre), len(post)

(3994, 6351)

Define a simple function to check for possible pairs:
- check that the distance between the boresight intercepts is < half the first image width
- put some constraints on the phase angle and and field of view etc.

In [9]:
def best_match(df1, df2, idx1):
    matches = []
    distance = []
    img = df1.loc[idx1]
    for idx, match in df2.iterrows():
        
        dist = math.sqrt( (match.surf_int_x-img.surf_int_x)**2 + (match.surf_int_y-img.surf_int_y)**2 + (match.surf_int_z-img.surf_int_z)**2 )
        phase_diff = abs(img.phase_angle-match.phase_angle)
        alt_diff = abs(img.sc_altitude-match.sc_altitude)
        
        # vec1 = np.array([match.sc_position_x, match.sc_position_y, match.sc_position_z])
        # vec2 = np.array([img.sc_position_x, img.sc_position_y, img.sc_position_z])
        # vector_diff = np.rad2deg(np.arccos( np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2))))
        
        if (dist < img.half_width/2.) and (alt_diff<5.) and (phase_diff<30.):
            distance.append(dist)
            matches.append(idx)
            
    if len(matches)>0:
        return matches[distance.index(min(distance))], min(distance)
    else:
        return None, None

This now returns the index of the best match and the distance between the intercept point on the comet

In [10]:
idx, dist = best_match(pre, post, 200)
idx, dist

(7362, 0.49732685429202395)

Now let's look at these two images - we will use the browse image link to display the jpeg image streamed from the archive:

In [11]:
def show_pair(df1, df2, idx1, idx2=None):

    if idx2 is None:
        idx2, dist = best_match(df1, df2, idx1)
        if idx2 is None:
            print('No match found')
            return

    img1 = display.Image(df1.loc[idx1].browse_url)
    img2 = display.Image(df2.loc[idx2].browse_url)
    wi1 = widgets.Image(value=img1.data, format='jpg', width=400, height=400)
    wi2 = widgets.Image(value=img2.data, format='jpg', width=400, height=400)
    pair = widgets.HBox([wi1, wi2])
    display.display(pair)

In [12]:
show_pair(pre, post, 200)

Loop through all pre-perihelion images and find the best post-perihleion image, if any.

Note that this is not at all optimised, and running all the combinations is going to be slooow. 

In [13]:
match_pre = []
match_post = []

In [14]:
for idx1 in pre.index:
    idx2, dist = best_match(pre, post, idx1)
    if idx2 is None:
        continue
    else:
        match_pre.append(idx1)
        match_post.append(idx2)

In [15]:
len(match_pre)

3405

In [16]:
idx = 500
show_pair(pre, post, match_pre[idx], match_post[idx])

In [17]:
idx = 5
show_pair(pre, post, match_pre[idx], match_post[idx])